Übung : Einfaches Image-Management
-------------------------------------------------------

Erstellt einen Account auf https://hub.docker.com/.

Erstellt, auf http://localhost:32188/ ein neues `Terminal` rechts oben mit `New` und logt Euch auf Docker Hub ein:

    docker login

Tagt die zwei vorhanden Images mit Eurem Namen und pusht diese in auf Docker Hub

z.B.:

    docker tag hello-world <username>/hello-world
    docker push <username>hello-world


In [ ]:
! docker ...

Docker Images als im «tar»-Format speichern und laden

In [ ]:
! docker save hello-world -o hello-world.tar
! docker rmi hello-world
! docker image ls

In [ ]:
! ls -ls *.tar

In [ ]:
! docker load -i hello-world.tar
! docker image ls